In [1]:
import os
import openai
import json
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import time


# Connect to ChatGpt Api

In [2]:
openai.organization = os.getenv("CHATGPTORG")
openai.api_key = os.getenv("CHATGPTKEY")
openai.Model.list()

<OpenAIObject list at 0x1374dbea0> JSON: {
  "data": [
    {
      "created": 1677532384,
      "id": "whisper-1",
      "object": "model",
      "owned_by": "openai-internal",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1683912666,
          "group": null,
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "whisper-1"
    },
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
         

## Check if the connection and saving is working


In [3]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

<OpenAIObject chat.completion id=chatcmpl-7YEcS6qUFKnyvb40wrw3hElDhwGFa at 0x1374e66d0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The World Series in 2020 was played at the Globe Life Field in Arlington, Texas.",
        "role": "assistant"
      }
    }
  ],
  "created": 1688393704,
  "id": "chatcmpl-7YEcS6qUFKnyvb40wrw3hElDhwGFa",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 53,
    "total_tokens": 72
  }
}

## Start testing

In [4]:
# open our test dataframe from the csv
df = pd.read_csv("data/test_datasets/random/random_sample_100_v2.csv")

In [5]:
df.shape

(100, 23)

In [7]:
# create a new column for the chatbot response
df['chatbot_response'] = ''
df['chatbot_response_raw'] = ''
df['chatbot_question'] = ''

In [14]:
# loop through the dataframe and send the messages to the chatbot
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    message = [
        {"role": "system", "content": "Please provide a category for the following products: Like watches or electronics. Provide a category for each product. Use these categories: Electronics, Home Appliances, Fashion and Apparel, Beauty and Personal Care, Sports and Fitness, Home Decor and Furniture, Books and Media, Health and Wellness, Toys and Games, Automotive"},
        {"role": "user",
            "content": f" product 1: {row['title_left']}, product 2: {row['title_right']}"},
    ]
    error = ""
    for i in range(3):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=message,
                max_tokens=15,
            )
            # if the above call succeeds, break out of the loop and continue
            break
        except Exception as e:
            error = e
            print(f"Error: {e}")
            # if the call fails, wait for one minute before retrying
            time.sleep(60)

    # check if the response was set successfully
    if response:
        print(response['choices'][0]['message']['content'])
        df.at[index, 'chatbot_response_raw'] = response
        df.at[index, 'chatbot_response'] = response['choices'][0]['message']['content']
        df.at[index, 'chatbot_question'] = message
    else:
        df.at[index, 'chatbot_response_raw'] = error
        df.at[index, 'chatbot_response'] = -1
        df.at[index, 'chatbot_question'] = message
        print("Error: response was not set successfully")

  0%|          | 0/100 [00:00<?, ?it/s]

Product 1: Maxxis Maxxis Minion DHR2 29
product 1: Electronics
product 2: Electronics
Product 1: Electronics
Product 2: Electronics
Product 1: Electronics
Product 2: Electronics
product 1: Electronics
product 2: Electronics
product 1: Tissot Seastar 1000 T120
Product 1: Samsung 970 EVO SSD M.2 228
product 1: Books and Media
product 2: Electronics
Product 1: Electronics - iiyama 27' ETE,
Product 1: Maxxis High Roller II Dual Compound Tire (EXO
product 1: Electronics
product 2: Electronics
Product 1: Electronics (Printer ink cartridge)
Product 2: Electronics
Product 1: Electronics
Product 2: Electronics
Product 1: TISSOT SEASTAR 1000 AUTOMATIC
product 1: Electronics
product 2: Automotive
Product 1: RAM-B-408-75-1-UN7
Product 1: Electronics (Headphones)
Product 2: Electronics (
Product 1: Electronics 
Product 2: Electronics
Product 1: Electronics (specifically computer hardware)
Product 2:
product 1: Home Appliances
product 2: Home Appliances
product 1: Electronics (USB flash drive)
produ

KeyboardInterrupt: 

In [15]:
df.head()

,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,specTableContent_left,cluster_id_left,id_right,brand_right,...,pair_id,label,is_hard_negative,roberta-base_logits,roberta-base_prediction,rsupcon-base_logits,rsupcon-base_prediction,chatbot_response_raw,chatbot_response,chatbot_question
0,70108616,Maxxis,Maxxis Maxxis Minion DHR2 29 x 2.3 Folding Bead,The new incarnation of the Minion DHR. Ready t...,110.00,CAD,NaN,1084991,45040021,Maxxis,...,70108616#45040021,1,False,"[-4.3633804321, 4.6556377411]",1,[1.0],1,{'id': 'chatcmpl-7YEmCMSzYThaFhIvVXtbVGoDGIz4t...,Product 1: Maxxis Maxxis Minion DHR2 29,"[{'role': 'system', 'content': 'Please provide..."
1,67977820,NaN,Canon CLI251XL High Yield Black Inkjet Cartrid...,Crazy Inkjets is a leading supplier of high qu...,6.95,USD,NaN,767463,69787972,Samsung,...,67977820#69787972,0,True,"[3.8366084099, -4.2942962646]",0,[0.0],0,{'id': 'chatcmpl-7YEmDvlozsszS3QhJUr0Fx9fOrB7z...,product 1: Electronics\nproduct 2: Electronics,"[{'role': 'system', 'content': 'Please provide..."
2,79859336,Crucial,Crucial Crucial Memory 4GB DDR4 2666 Unbuffere...,NaN,34.99,CAD,NaN,1892167,65646040,Crucial,...,79859336#65646040,1,False,"[-4.363576889, 4.6508393288]",1,[1.0],1,{'id': 'chatcmpl-7YEmEkHHHFu6y6d4vT1IKP8KrTD5R...,Product 1: Electronics\nProduct 2: Electronics,"[{'role': 'system', 'content': 'Please provide..."
3,30368884,NaN,"Crucial MX500 250GB 2.5"" SATA III","MX500 250GB SATA 2.5-inch, SATA 6.0Gb/s, 560 M...",43.59,EUR,NaN,672125,86893846,NaN,...,30368884#86893846,0,False,"[3.8347194195, -4.2961273193]",0,[0.0],0,{'id': 'chatcmpl-7YEmGrZkC9rkTrVFa9LswBSORoybw...,Product 1: Electronics\nProduct 2: Electronics,"[{'role': 'system', 'content': 'Please provide..."
4,82078171,NaN,Intel Core I3 7th Gen 7100 3.90 Ghz; 2 Core 4 ...,"ntel i3-7100, Core. Processor family: 7th gen ...",2.899E3,ZAR,NaN,443612,6914049,WESTERN DIGITAL,...,82078171#6914049,0,False,"[3.8386721611, -4.3010449409]",0,[0.0],0,{'id': 'chatcmpl-7YEnskoAFTMcfjcniepMjp8XMsy6f...,product 1: Electronics\nproduct 2: Electronics,"[{'role': 'system', 'content': 'Please provide..."


In [16]:
# get one chatresponseraw
df['chatbot_response_raw'][0]

<OpenAIObject chat.completion id=chatcmpl-7YEmCMSzYThaFhIvVXtbVGoDGIz4t at 0x1376b9360> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "message": {
        "content": "Product 1: Maxxis Maxxis Minion DHR2 29",
        "role": "assistant"
      }
    }
  ],
  "created": 1688394308,
  "id": "chatcmpl-7YEmCMSzYThaFhIvVXtbVGoDGIz4t",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 15,
    "prompt_tokens": 135,
    "total_tokens": 150
  }
}